In [3]:
import pandas as pd
import os

In [4]:
def getTripsFrame():
    data = pd.read_csv('../data/processed/trips_cleaned.csv',  index_col=0 )
    data['start_time'] = pd.to_datetime(data['start_time'])
    data['end_time'] = pd.to_datetime(data['end_time'])
    
    return data

In [5]:
weather = pd.read_csv('../data/external/weather.gz', compression='gzip')

In [6]:
weather

timestamp  temp_2m  humidity_2m  dew_point_2m  max_at_2m  \
0      2019-01-01 00:00:00      7.8         91.0           6.4        7.9   
1      2019-01-01 00:10:00      8.0         86.2           5.8        8.0   
2      2019-01-01 00:20:00      8.0         86.3           5.9        8.0   
3      2019-01-01 00:30:00      7.9         87.1           5.9        8.0   
4      2019-01-01 00:40:00      7.9         87.2           5.9        7.9   
...                    ...      ...          ...           ...        ...   
52555  2019-12-31 23:10:00      0.4        100.0           0.4        0.5   
52556  2019-12-31 23:20:00      0.0        100.0           0.0        0.3   
52557  2019-12-31 23:30:00     -0.1         99.7          -0.1        0.0   
52558  2019-12-31 23:40:00     -1.4         98.4          -1.6        0.0   
52559  2019-12-31 23:50:00     -1.8         99.7          -1.8       -0.4   

       mean_speed_h/s  direction_degree_x  max_m/s  min_mean_m/s  \
0                 7.4                 260     10.6           4.6   
1                 8.4                 260     11.5           5.1   
2                 6.5                 250      8.9           4.1   
3                 7.4                 250     11.5           4.6   
4                 7.0                 250      9.8           5.1   
...               ...                 ...      ...           ...   
52555             0.7                 310      1.0           0.0   
52556             0.5                 300      1.0           0.0   
52557             0.4                 320      0.5           0.0   
52558             0.8                 180      1.5           0.0   
52559             1.2                 190      2.1           0.5   

       max_mean_m/s  direction_degree_y  min    mm  
0               7.6                 260   10  0.07  
1               8.5                 250   10  0.01  
2               8.4                 260   10  0.00  
3               7.4                 260    9  0.00  
4               7.6                 260    0  0.00  
...             ...                 ...  ...   ...  
52555           0.7                 290    0  0.00  
52556           0.7                 310    0  0.00  
52557           0.5                 290    0  0.00  
52558           0.8                 180    0  0.00  
52559           1.2                 190    0  0.00  

[52560 rows x 13 columns]

In [7]:
weather['timestamp'] = pd.to_datetime(weather['timestamp'])

In [8]:
trips = getTripsFrame()

In [9]:
# Floor start time to next 10 minutes so we can merge it with weather data

trips['sTime_floored'] = pd.to_datetime(trips['start_time']).dt.floor('10T')
trips['sTime_floored'] = pd.to_datetime(trips['sTime_floored'])

In [10]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143665 entries, 20685 to 20751
Data columns (total 20 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   bike_type       143665 non-null  int64         
 1   identification  143665 non-null  int64         
 2   start_time      143665 non-null  datetime64[ns]
 3   end_time        143665 non-null  datetime64[ns]
 4   duration_sec    143665 non-null  float64       
 5   start_lng       143665 non-null  float64       
 6   start_lat       143665 non-null  float64       
 7   end_lng         143665 non-null  float64       
 8   end_lat         143665 non-null  float64       
 9   start_place     143665 non-null  int64         
 10  end_place       143665 non-null  int64         
 11  start_plz       143665 non-null  int64         
 12  end_plz         143665 non-null  int64         
 13  month           143665 non-null  int64         
 14  booking_date    143665 non-null  

In [11]:
data = trips.merge(right= weather, left_on='sTime_floored', right_on='timestamp', how='left')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143665 entries, 0 to 143664
Data columns (total 33 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   bike_type           143665 non-null  int64         
 1   identification      143665 non-null  int64         
 2   start_time          143665 non-null  datetime64[ns]
 3   end_time            143665 non-null  datetime64[ns]
 4   duration_sec        143665 non-null  float64       
 5   start_lng           143665 non-null  float64       
 6   start_lat           143665 non-null  float64       
 7   end_lng             143665 non-null  float64       
 8   end_lat             143665 non-null  float64       
 9   start_place         143665 non-null  int64         
 10  end_place           143665 non-null  int64         
 11  start_plz           143665 non-null  int64         
 12  end_plz             143665 non-null  int64         
 13  month               143665 no

In [13]:
data.drop(columns=['sTime_floored'], inplace=True)

In [14]:
data.to_csv('../data/processed/trips_weather.csv')
